<a href="https://colab.research.google.com/github/NeerajKhadagade29/Web-Scraping-JavaScript-Content/blob/main/Web_Scraping_JavaScript_Content_Full_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# For using dataframe
import pandas as pd

# For making HTTSP requests
import requests

# For web scraping
from bs4 import BeautifulSoup

# PART 1 

In [ ]:
sites = []
links = []
for num in range (1, 10, 1):
    url = 'https://www.lex100.com/firm-profiles/?sf_paged='+str(num)+'/'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    profiles = soup.find_all('div', class_ = 'firm-profile-full')
    for profile in profiles:
        link = profile.find('h2', class_ = 'firm-profile__title').find('a', href = True)['href']
        links.append(link)
        
        try:
            site_details = {
                'Name' : profile.find('h2', class_ = 'firm-profile__title').find('a').text,
                'Description' : profile.find('div', class_ = 'firm-profile__verdict').text.strip(),
                'Awards/Recognition' : profile.find('div', id = 'profile-page-links-list').text.strip()
            }
            sites.append(site_details)

        except:
            site_details = {
                'Name' : profile.find('h2', class_ = 'firm-profile__title').find('a').text,
                'Description' : "Description Not Found",
                'Awards/Recognition' : "Awards/Recognition not found"
            }
            
            sites.append(site_details)
    

In [ ]:
links

['https://www.lex100.com/profiles/addleshaw-goddard/',
 'https://www.lex100.com/profiles/akin-gump-strauss-hauer-feld/',
 'https://www.lex100.com/profiles/ashfords-llp/',
 'https://www.lex100.com/profiles/ashurst/',
 'https://www.lex100.com/profiles/b-p-collins-llp/',
 'https://www.lex100.com/profiles/baker-mckenzie/',
 'https://www.lex100.com/profiles/bdb-pitmans/',
 'https://www.lex100.com/profiles/bevan-brittan-llp/',
 'https://www.lex100.com/profiles/bird-bird-llp/',
 'https://www.lex100.com/profiles/birketts-llp/',
 'https://www.lex100.com/profiles/blaser-mills-law/',
 'https://www.lex100.com/profiles/blm/',
 'https://www.lex100.com/profiles/boodle-hatfield-llp/',
 'https://www.lex100.com/profiles/brabners-llp/',
 'https://www.lex100.com/profiles/bristows-llp/',
 'https://www.lex100.com/profiles/bryan-cave-leighton-paisner-llp/',
 'https://www.lex100.com/profiles/burges-salmon/',
 'https://www.lex100.com/profiles/burness-paull-llp/',
 'https://www.lex100.com/profiles/capsticks-llp

In [ ]:
profiles = pd.DataFrame(sites)
profiles.drop_duplicates(inplace = True)
profiles

,Name,Description,Awards/Recognition
0,Addleshaw Goddard,Trainees looking for a ‘friendly and inclusive...,
1,Akin Gump Strauss Hauer & Feld,"Akin Gump Strauss Hauer & Feld LLP, a leading ...",Akin Gump Strauss Hauer & Feld is a Lex 100 Wi...
2,Ashfords LLP,Description Not Found,Awards/Recognition not found
3,Ashurst,Description Not Found,Awards/Recognition not found
4,B P Collins LLP,Description Not Found,Awards/Recognition not found
...,...,...,...
115,Wilkin Chapman LLP,Description Not Found,Awards/Recognition not found
116,Withers LLP,Description Not Found,Awards/Recognition not found
117,Womble Bond Dickinson,Description Not Found,Awards/Recognition not found
118,Woodfines LLP,Description Not Found,Awards/Recognition not found


### This is the end of Part 1. Please refer to Part 2 for the rest of the code.

___
# PART 2

In [ ]:
# Importing the Selenium libraries
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

data = {}
scores = pd.DataFrame(columns = ['Name','Address'])

for link in links[:5]:
    # Extract the scores
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
#     options.add_argument('--disable-gpu')
    path = "C:\\Users\\lenovo\\Documents\\chromedriver.exe"
    driver = webdriver.Chrome(path, options = options)
    driver.get(link)
    
    # Fetching the score from the JavaScript Tag
    metric_values = []
    results_vals = driver.execute_script("""""
                        let results_vals = chart.data.map(({result})=>result);
                        console.log(results_vals);
                        return results_vals
                        """"")
    metric_values.append(results_vals)
    
    # Extracting info
    soup = requests.get(link)
    webpage = BeautifulSoup(soup.content, 'html.parser')

    data['Name'] = webpage.find('div', class_ = 'letterhead').find('h1', class_ = 'styled').get_text()

    try:
        data['Address'] = webpage.find('div', class_ = 'contact-box').find('div', class_ = 'address contact').text.rstrip()
    except:
        data['Address'] = "Not Found"

    # Satisfaction Scores
    try:
        data['Satisfaction'] =''.join([score[0].strip() for score in metric_values])
    except:
        data['Satisfaction'] = "Not Found"
    # Expectations Scores
    try:
        data['Expectations'] =''.join([score[1].strip() for score in metric_values])
    except:
        data['Expectations'] = "Not Found"
    # Work Quality Scores
    try:
        data['Work Quality'] =''.join([score[2].strip() for score in metric_values])
    except:
        data['Work Quality'] = "Not Found"

    # Client Contact Scores
    try:
        data['Client Contact'] =''.join([score[3].strip() for score in metric_values])
    except:
        data['Client Contact'] = "Not Found"
    # Work/Life Scores
    try:
        data['Work/Life'] =''.join([score[4].strip() for score in metric_values])
    except:
        data['Work/Life'] = "Not Found"
    # Social Life Scores
    try:
        data['Social Life'] =''.join([score[5].strip() for score in metric_values])
    except:
        data['Social Life'] = "Not Found"
    # Vac. Scheme Scores
    try:
        data['Vac. Scheme'] =''.join([score[6].strip() for score in metric_values])
    except:
        data['Vac. Scheme'] = "Not Found"
    # Confidence Scores
    try:
        data['Confidence'] =''.join([score[7].strip() for score in metric_values])
    except:
        data['Confidence'] = "Not Found"
    # Renumeration Scores
    try:
        data['Renumeration'] =''.join([score[8].strip() for score in metric_values])
    except:
        data['Renumeration'] = "Not Found"
    # Inclusiveness Scores
    try:
        data['Inclusiveness'] =''.join([score[9].strip() for score in metric_values])
    except:
        data['Inclusiveness'] = "Not Found"
    # International Scores
    try:
        data['International'] =''.join([score[10].strip() for score in metric_values])
    except:
        data['International'] = "Not Found"
    # Approachability Scores
    try:
        data['Approachability'] =''.join([score[11].strip() for score in metric_values])
    except:
        data['Approachability'] = "Not Found"
        
    scores = scores.append(data, ignore_index=True)
#    print("Scores Extarcted")


[['79.5', '83.9', '83.6', '74.4', '75.1', '78.5', '90.0', '76.6', '66.9', '82.6', '68.0', '89.8']]
[['82.3', '90.3', '88.3', '70.6', '70.3', '54.3', '88.3', '81.3', '96.3', '84.3', '83.3', '80.3']]
[['83.2', '85.9', '81.4', '89.5', '86.8', '76.8', '76.6', '81.4', '78.6', '86.8', '25.3', '87.7']]
[['76.0', '79.6', '77.5', '71.0', '59.3', '69.3', '80.0', '78.2', '74.3', '75.7', '77.8', '79.6']]
[['83.1', '81.1', '85.1', '77.1', '83.1', '57.1', '77.1', '75.1', '55.1', '77.1', '83.1']]


In [ ]:
scores

,Name,Address,Approachability,Client Contact,Confidence,Expectations,Inclusiveness,International,Renumeration,Satisfaction,Social Life,Vac. Scheme,Work Quality,Work/Life
0,Addleshaw Goddard,"Milton Gate, 60 Chiswell Street, London EC1Y 4AG",89.8,74.4,76.6,83.9,82.6,68.0,66.9,79.5,78.5,90.0,83.6,75.1
1,Akin Gump Strauss Hauer & Feld,"Ten Bishops Square, Eighth Floor, London E1 6EG",80.3,70.6,81.3,90.3,84.3,83.3,96.3,82.3,54.3,88.3,88.3,70.3
2,Ashfords LLP,Not Found,87.7,89.5,81.4,85.9,86.8,25.3,78.6,83.2,76.8,76.6,81.4,86.8
3,Ashurst,Not Found,79.6,71.0,78.2,79.6,75.7,77.8,74.3,76.0,69.3,80.0,77.5,59.3
4,B P Collins LLP,Not Found,Not Found,77.1,75.1,81.1,77.1,83.1,55.1,83.1,57.1,77.1,85.1,83.1


In [ ]:
final_df = pd.merge(profiles, scores, left_on='Name', right_on='Name')
final_df.head(5)

,Name,Description,Awards/Recognition,Address,Approachability,Client Contact,Confidence,Expectations,Inclusiveness,International,Renumeration,Satisfaction,Social Life,Vac. Scheme,Work Quality,Work/Life
0,Addleshaw Goddard,Trainees looking for a ‘friendly and inclusive...,,"Milton Gate, 60 Chiswell Street, London EC1Y 4AG",89.8,74.4,76.6,83.9,82.6,68.0,66.9,79.5,78.5,90.0,83.6,75.1
1,Akin Gump Strauss Hauer & Feld,"Akin Gump Strauss Hauer & Feld LLP, a leading ...",Akin Gump Strauss Hauer & Feld is a Lex 100 Wi...,"Ten Bishops Square, Eighth Floor, London E1 6EG",80.3,70.6,81.3,90.3,84.3,83.3,96.3,82.3,54.3,88.3,88.3,70.3
2,Ashfords LLP,Description Not Found,Awards/Recognition not found,Not Found,87.7,89.5,81.4,85.9,86.8,25.3,78.6,83.2,76.8,76.6,81.4,86.8
3,Ashurst,Description Not Found,Awards/Recognition not found,Not Found,79.6,71.0,78.2,79.6,75.7,77.8,74.3,76.0,69.3,80.0,77.5,59.3
4,B P Collins LLP,Description Not Found,Awards/Recognition not found,Not Found,Not Found,77.1,75.1,81.1,77.1,83.1,55.1,83.1,57.1,77.1,85.1,83.1


final_df.to_csv('companies_data.csv', index = False)

___

The code ends here. Please feel free to drop any suggestions.

Thank you! :)
